In [46]:
#michael is sus

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as stat

In [47]:
company_tweet = pd.read_csv('Data/Tweets/Company_Tweet.csv')
tweet = pd.read_csv('Data/Tweets/Tweet.csv')
company = pd.read_csv('Data/Tweets/Company.csv')
company_value = pd.read_csv('Data/CompanyStocks/CompanyValues.csv')

In [48]:
tweet = tweet.dropna()
#merge company tweets and tweets on id
tweet_df = pd.merge(company_tweet , tweet , on="tweet_id", how= "inner")

In [49]:
#create variable 'total_engagement'
tweet_df["total_engangement"] = tweet_df["comment_num"] + tweet_df["retweet_num"] + tweet_df["like_num"]
#remove tweets below certain engagement
engagement_threshold = 40
tweet_df = tweet_df[tweet_df["total_engangement"] > engagement_threshold]
tweet_df.head()
print('tweet count: ', tweet_df.size)

tweet count:  485118


In [50]:
#convert original time on tweet_df to date time
tweet_df['post_date'] = pd.to_datetime(tweet_df['post_date'], unit='s')
tweet_df['day_date'] = pd.to_datetime(tweet_df['post_date'].apply(lambda date: date.date()))

In [51]:
#order chronilogically
tweet_df = tweet_df.sort_values(by="day_date")

In [52]:
#convert time value on company stock data to date time
stock_df = company_value
stock_df['day_date']  = pd.to_datetime(stock_df['day_date'])
stock_df.sample(5)


,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value
9675,GOOG,2018-09-26,1180.49,1346434,1185.15,1194.23,1174.765
709,AAPL,2017-08-03,155.57,26604140,157.05,157.21,155.020
741,AAPL,2017-06-19,146.34,32203640,143.66,146.74,143.660
15798,TSLA,2015-02-09,217.48,3469776,215.38,217.93,211.990
2926,AAPL,2018-08-11,207.53,24592460,207.36,209.10,206.670


In [53]:
#drop un-needed features
tweet_df = tweet_df.drop(['tweet_id', 'post_date', 'comment_num', 'retweet_num', 'like_num'], axis=1)

In [54]:
#remove any stock data that is outside of the tweet data
start_day = min(tweet_df['day_date'])
print(f"Start day: {start_day}")

end_day = max(tweet_df['day_date'])
print(f"End day: {end_day}")

Start day: 2015-01-01 00:00:00
End day: 2019-12-31 00:00:00


In [55]:
stock_df = stock_df[(stock_df['day_date'] >= start_day) & (stock_df['day_date'] <= end_day)]

In [56]:
#sort stock data chronilogically
stock_df = stock_df.sort_values(by="day_date")
stock_df.head()

,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value
2517,AAPL,2015-01-01,110.38,41304780,112.82,113.13,110.21
10810,GOOG,2015-01-01,526.40,1367110,531.25,532.60,525.80
8687,GOOGL,2015-01-01,530.66,1235889,537.74,538.40,530.20
16960,TSLA,2015-01-01,222.41,2392947,223.09,225.68,222.25
5602,AMZN,2015-01-01,310.35,2048676,311.55,312.98,310.01


In [57]:
tweet_df.head()

,ticker_symbol,writer,body,total_engangement,day_date
40,AAPL,CNBC,This is Wall Street's top pick in 2015. Hint: ...,64,2015-01-01
47,AAPL,WSJ,"See how tech companies like Apple, Twitter and...",347,2015-01-01
239,AAPL,greatspoke,Top Stock Analyst Releases Report on Top Stock...,433,2015-01-01
240,AAPL,greatspoke,Analyst Report on Top Stock Market Options for...,433,2015-01-01
241,AMZN,greatspoke,Analyst Report on Top Stock Market Options for...,433,2015-01-01


In [58]:
APPL_df = stock_df.loc[stock_df['ticker_symbol'] == 'AAPL']

GOOG_df = stock_df.loc[stock_df['ticker_symbol'] == 'GOOG']
GOOGL_df = stock_df.loc[stock_df['ticker_symbol'] == 'GOOGL']


TSLA_df = stock_df.loc[stock_df['ticker_symbol'] == 'TSLA']
AMZN_df = stock_df.loc[stock_df['ticker_symbol'] == 'AMZN']
MSFT_df = stock_df.loc[stock_df['ticker_symbol'] == 'MSFT']

In [59]:
#there are two types of google stocks, one has voting power in the company while the other doesnt
GOOGL_df.head()

,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value
8687,GOOGL,2015-01-01,530.66,1235889,537.74,538.4000,530.20
7530,GOOGL,2015-01-02,529.55,1325271,532.60,535.8000,527.88
8688,GOOGL,2015-01-03,529.55,1325271,532.60,535.8000,527.88
8689,GOOGL,2015-01-04,529.55,1325271,532.60,535.8000,527.88
7529,GOOGL,2015-01-05,519.46,2054120,527.15,527.9899,517.75


In [60]:
APPL_tweet = tweet_df.loc[tweet_df['ticker_symbol'] == 'AAPL']

In [61]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [62]:
import nltk                             
# from nltk.corpus import twitter_samples                            
# from nltk.corpus import stopwords 
# from nltk.stem import PorterStemmer
# from nltk.tokenize import TweetTokenizer  

import random  
import re                     
import string


In [63]:
def remove_special_character(tweet):
#     print(tweet)
    # remove the old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)

    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)

    # remove hashtags. only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)

    # remove single numeric terms in the tweet. 
    tweet = re.sub(r'[0-9]', '', tweet)
    
    return tweet

In [64]:
tweet_df["body"] = tweet_df["body"].apply(lambda tweet: remove_special_character(tweet))
tweet_df["body"] = tweet_df["body"].str.lower()
tweet_df.sample(5)

# ?? why is the body text so fucked up ??

,ticker_symbol,writer,body,total_engangement,day_date
3713184,TSLA,orthereaboot,details in this $tsla story must all be read. ...,49,2018-06-01
4246304,TSLA,GerberKawasaki,it’s possible that tesla just expanded the mar...,235,2019-11-22
1362371,AAPL,DanielEran,apple music hits record.m streams in hours. s...,129,2018-06-30
2426824,AMZN,Sandro_power,waiting for confirmation in this chop has lead...,157,2018-06-25
3882781,TSLA,ValueAnalyst1,if you're surprised by macquarie's $ price tar...,69,2018-10-08


In [65]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [66]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vanderSentimentAnalyzer = SentimentIntensityAnalyzer()

In [67]:
def getVanderScore(tweet):    
    vs = vanderSentimentAnalyzer.polarity_scores(tweet)
    score = vs['compound']
    return score

In [68]:
tweet_df['vander_score'] = tweet_df['body'].apply(lambda tweet: getVanderScore(tweet))


In [69]:
#create a weighted scored based off engagement and sentiment analysis score
tweet_df['summarative_score'] = tweet_df['total_engangement'] * tweet_df['vander_score'] * 0.01
tweet_df = tweet_df.drop(['total_engangement', 'vander_score'], axis=1)


In [70]:
#sum or avg the summarative_scores for one date
APPL_tweet = tweet_df.loc[tweet_df['ticker_symbol'] == 'AAPL']

GOOG_tweet = tweet_df.loc[tweet_df['ticker_symbol'] == 'GOOG']
GOOGL_tweet = tweet_df.loc[tweet_df['ticker_symbol'] == 'GOOGL']


TSLA_tweet = tweet_df.loc[tweet_df['ticker_symbol'] == 'TSLA']
AMZN_tweet = tweet_df.loc[tweet_df['ticker_symbol'] == 'AMZN']
MSFT_tweet = tweet_df.loc[tweet_df['ticker_symbol'] == 'MSFT']

In [71]:
GOOG_tweet.head()

,ticker_symbol,writer,body,day_date,summarative_score
1850635,GOOG,WSJ,google now controls or occupies a strip of man...,2015-01-04,0.000000
4040,GOOG,DanielEran,"despite lawsuit, $aapl ios is far more storag...",2015-01-05,0.410522
1854377,GOOG,WSJ,google is exploring how to use frequencies to ...,2015-01-20,0.000000
1854055,GOOG,WSJ,google is close to investing $ billion in spac...,2015-01-20,1.547000
1854431,GOOG,The_Real_Fly,"obama is long $ebay, $goog and $tsla",2015-01-21,0.000000


In [72]:
APPL_tweet = APPL_tweet.groupby('day_date')['summarative_score'].sum()
GOOG_tweet = GOOG_tweet.groupby('day_date')['summarative_score'].sum()
GOOGL_tweet = GOOGL_tweet.groupby('day_date')['summarative_score'].sum()
TSLA_tweet = TSLA_tweet.groupby('day_date')['summarative_score'].sum()
AMZN_tweet = AMZN_tweet.groupby('day_date')['summarative_score'].sum()
MSFT_tweet = MSFT_tweet.groupby('day_date')['summarative_score'].sum()


In [73]:
APPL_df = APPL_df.merge(APPL_tweet, how = 'outer', on = 'day_date')
GOOG_df = APPL_df.merge(GOOG_tweet, how = 'outer', on = 'day_date')
GOOGL_df = APPL_df.merge(GOOGL_tweet, how = 'outer', on = 'day_date')
TSLA_df = APPL_df.merge(TSLA_tweet, how = 'outer', on = 'day_date')
AMZN_df = APPL_df.merge(AMZN_tweet, how = 'outer', on = 'day_date')
MSFT_df = APPL_df.merge(MSFT_tweet, how = 'outer', on = 'day_date')

APPL_df.head()

,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value,summarative_score
0,AAPL,2015-01-01,110.38,41304780,112.82,113.13,110.21,3.911989
1,AAPL,2015-01-02,109.33,53143770,111.39,111.44,107.35,2.044350
2,AAPL,2015-01-03,109.33,53143770,111.39,111.44,107.35,NaN
3,AAPL,2015-01-04,109.33,53143770,111.39,111.44,107.35,1.740136
4,AAPL,2015-01-05,106.25,64210880,108.29,108.65,105.41,0.866415


In [74]:
APPL_df.replace(to_replace = np.nan, value = 0)
GOOG_df.replace(to_replace = np.nan, value = 0)
GOOGL_df.replace(to_replace = np.nan, value = 0)
AMZN_df.replace(to_replace = np.nan, value = 0)
TSLA_df.replace(to_replace = np.nan, value = 0)
MSFT_df.replace(to_replace = np.nan, value = 0)


,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value,summarative_score_x,summarative_score_y
0,AAPL,2015-01-01,110.38,41304780,112.82,113.13,110.21,3.911989,0.000000
1,AAPL,2015-01-02,109.33,53143770,111.39,111.44,107.35,2.044350,0.000000
2,AAPL,2015-01-03,109.33,53143770,111.39,111.44,107.35,0.000000,0.000000
3,AAPL,2015-01-04,109.33,53143770,111.39,111.44,107.35,1.740136,0.000000
4,AAPL,2015-01-05,106.25,64210880,108.29,108.65,105.41,0.866415,0.410522
...,...,...,...,...,...,...,...,...,...
1821,AAPL,2019-12-27,289.80,36592940,291.12,293.97,288.12,0.115701,1.539251
1822,AAPL,2019-12-28,289.80,36592940,291.12,293.97,288.12,0.411740,0.116788
1823,AAPL,2019-12-29,289.80,36592940,291.12,293.97,288.12,2.102808,2.693193
1824,AAPL,2019-12-30,291.52,36059610,289.46,292.69,285.22,7.807207,0.000000


# # LSTM MODEL: 

In [75]:
#pip install tensorflow==2.9.1


In [76]:
#here we will do the LSTM model

#/-\|/-\|
import math
import tensorflow 
from tensorflow import keras as ks
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Dropout, LSTM
from keras.layers import Dense, LSTM

from keras.optimizers import Adam

plt.style.use('fivethirtyeight')


In [77]:
APPL_df = APPL_df.fillna(0)
APPL_df.head(10)

,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value,summarative_score
0,AAPL,2015-01-01,110.38,41304780,112.82,113.13,110.210,3.911989
1,AAPL,2015-01-02,109.33,53143770,111.39,111.44,107.350,2.044350
2,AAPL,2015-01-03,109.33,53143770,111.39,111.44,107.350,0.000000
3,AAPL,2015-01-04,109.33,53143770,111.39,111.44,107.350,1.740136
4,AAPL,2015-01-05,106.25,64210880,108.29,108.65,105.410,0.866415
5,AAPL,2015-01-06,106.26,65682250,106.54,107.43,104.630,0.000000
6,AAPL,2015-01-07,107.75,39983350,107.20,108.20,106.695,-0.120208
7,AAPL,2015-01-08,111.89,59168760,109.23,112.15,108.700,0.887594
8,AAPL,2015-01-09,112.01,53645730,112.67,113.25,110.210,0.000000
9,AAPL,2015-01-10,112.01,53645730,112.67,113.25,110.210,0.000000


In [78]:
forecast_out= 3
x1 = APPL_df.iloc[: , 3:]
x3 = APPL_df['PrevNClose'] = APPL_df[['close_value']].shift(forecast_out)
print(x3.shape)
print(x3)


X = pd.concat([x1,x3] , axis = 1)
X.pop("close_value")
#X = X.drop(labels = None, axis = 0, index = 'close_value')
X= X[forecast_out:]

print(X.shape)
print(X)

y = APPL_df[['close_value']]
y = y[forecast_out:]
print(y)

(1826, 1)
      close_value
0             NaN
1             NaN
2             NaN
3          110.38
4          109.33
...           ...
1821       284.27
1822       284.27
1823       289.91
1824       289.80
1825       289.80

[1826 rows x 1 columns]
(1823, 5)
        volume  open_value  high_value  low_value  summarative_score
3     53143770      111.39      111.44    107.350           1.740136
4     64210880      108.29      108.65    105.410           0.866415
5     65682250      106.54      107.43    104.630           0.000000
6     39983350      107.20      108.20    106.695          -0.120208
7     59168760      109.23      112.15    108.700           0.887594
...        ...         ...         ...        ...                ...
1821  36592940      291.12      293.97    288.120           0.115701
1822  36592940      291.12      293.97    288.120           0.411740
1823  36592940      291.12      293.97    288.120           2.102808
1824  36059610      289.46      292.69    285.220

In [79]:
#Split Train and Test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1]))

print(x_train.shape)



(1458, 5)


In [80]:
#Scale Features: https://towardsdatascience.com/what-and-why-behind-fit-transform-vs-transform-in-scikit-learn-78f915cf96fe
scaler = MinMaxScaler(feature_range = (0,1))
x_train = scaler.fit_transform(x_train)

print(x_train)


[[0.41983046 0.2066428  0.19738013 0.1912089  0.32252406]
 [0.09712986 0.02729714 0.02370242 0.02671411 0.30643867]
 [0.39344474 0.3334825  0.32733564 0.30192462 0.31454042]
 ...
 [0.10110693 0.50303302 0.49594661 0.50005012 0.3198676 ]
 [0.06080742 0.32851034 0.32491349 0.32924018 0.31507312]
 [0.3176094  0.09561456 0.10123579 0.09372494 0.3396114 ]]


In [81]:
scaler2 = MinMaxScaler().fit(x_test)
#print(scaler2)
#print(scaler2.data_min_)
#print(scaler2.feature_range)
x_test = scaler2.transform(x_test)

print(x_test)

[[0.0649375  0.29755672 0.2907569  0.29636839 0.5051896 ]
 [0.12502528 0.04753105 0.03847511 0.04527993 0.4516996 ]
 [0.12586565 0.37275434 0.37189249 0.37361061 0.51879139]
 ...
 [0.25711298 0.06349451 0.06091473 0.06095375 0.47735509]
 [0.20007031 0.20316189 0.19620796 0.20201813 0.47735509]
 [0.14085687 0.6592239  0.65195905 0.64366132 0.597537  ]]


In [82]:

model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = ( x_train.shape[1],1 )))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))


#model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
#model.add(Dense(1))
#model.compile(optimizer='adam', loss='mse')



In [83]:
#Compile model
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

#Train model
#xt_small = x_train[:1000]
#yt_small = y_train[:1000]
#model.fit(x_train, y_train, batch_size =1 , epochs = 3)
model.fit(x_train, y_train, batch_size =1 , epochs = 10)


Epoch 1/10
1458/1458 [==============================] - 4s 2ms/step - loss: 2594.4519
Epoch 2/10
1458/1458 [==============================] - 2s 2ms/step - loss: 26.5219
Epoch 3/10
1458/1458 [==============================] - 2s 2ms/step - loss: 15.3390
Epoch 4/10
1458/1458 [==============================] - 2s 2ms/step - loss: 13.0718
Epoch 5/10
1458/1458 [==============================] - 2s 2ms/step - loss: 14.4478
Epoch 6/10
1458/1458 [==============================] - 2s 2ms/step - loss: 9.9245
Epoch 7/10
1458/1458 [==============================] - 2s 2ms/step - loss: 12.0595
Epoch 8/10
1458/1458 [==============================] - 2s 2ms/step - loss: 8.3023
Epoch 9/10
1458/1458 [==============================] - 2s 2ms/step - loss: 7.3430
Epoch 10/10
1458/1458 [==============================] - 2s 2ms/step - loss: 7.2350


In [84]:
#Test Dataset
#x_test= np.array(x_test)
#x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1]))
print(x_test)
print(x_test.shape)


predictions = model.predict(x_test)
#print(predictions.shape)

print(predictions)

#pred = np.zeros(predictions.shape)
#pred = np.array(pred)
#pred = np.concatenate( [pred, pred, pred, pred, predictions] , axis = 1)

#predictions = scaler.inverse_transform(predictions)
#predictions = scaler.inverse_transform(predictions)
#predictions = predictions[:, 4]
#print(predictions)

#predictions = np.reshape(predictions, (y_test.shape[0], y_test.shape[1]))


[[0.0649375  0.29755672 0.2907569  0.29636839 0.5051896 ]
 [0.12502528 0.04753105 0.03847511 0.04527993 0.4516996 ]
 [0.12586565 0.37275434 0.37189249 0.37361061 0.51879139]
 ...
 [0.25711298 0.06349451 0.06091473 0.06095375 0.47735509]
 [0.20007031 0.20316189 0.19620796 0.20201813 0.47735509]
 [0.14085687 0.6592239  0.65195905 0.64366132 0.597537  ]]
(365, 5)
12/12 [==============================] - 1s 1ms/step
[[153.44536 ]
 [100.6794  ]
 [169.52762 ]
 [247.31453 ]
 [ 99.07609 ]
 [165.71384 ]
 [290.66968 ]
 [212.78305 ]
 [222.1732  ]
 [134.07729 ]
 [118.20917 ]
 [156.29805 ]
 [ 99.903496]
 [180.68861 ]
 [116.64232 ]
 [236.13422 ]
 [209.36421 ]
 [160.11752 ]
 [145.27502 ]
 [175.86302 ]
 [156.57556 ]
 [225.1938  ]
 [114.4746  ]
 [133.28473 ]
 [176.48117 ]
 [119.28124 ]
 [111.01346 ]
 [120.23252 ]
 [178.9067  ]
 [179.1313  ]
 [158.38997 ]
 [202.64075 ]
 [178.50583 ]
 [133.30733 ]
 [163.88637 ]
 [269.12766 ]
 [183.52017 ]
 [131.48462 ]
 [198.3397  ]
 [165.94165 ]
 [175.07935 ]
 [277.0500

In [85]:
rmse = np.sqrt( np.mean(( predictions - y_test) ** 2))
print(rmse)

close_value    5.568855
dtype: float64
